In [4]:
#pip install pingouin

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import datasets
import nilearn
from nilearn import masking
from nilearn import plotting
from nilearn import image
import os
import fsl.wrappers as fsl
import pandas as pd
os.environ['FSLDIR'] = '/usr/local/fsl'
os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'
from IPython.display import clear_output
from statsmodels.formula.api import ols
from IPython.display import display
from pingouin import ancova, anova, pairwise_tukey, pairwise_ttests, print_table
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns
import statsmodels.stats.multicomp as mc
import scipy.stats as stats
from termcolor import colored
import statsmodels.api as sm

In [2]:
pd.options.display.float_format = '{:,.4f}'.format
#data = pd.read_csv('manupulated_100_Agust4.csv').drop(columns='Unnamed: 0')
#data['SDB'].replace({'Pos': "Positive", 'Neg': "Negative"}, inplace=True)
#data['Group'].replace({0: "CN", 1: "MCI", 2: "AD"}, inplace=True)
#regions_F_dat = pd.read_csv("statistics_Overlap_21Sep.csv")
#data.head()
data = pd.read_csv

,RID,SubjectID,Sex,Weight,Group,SDB,Age,MMSCORE,MMDRAW,MMWRITE,...,vbm_Otha_R,vbm_cTtha_L,vbm_cTtha_R,vbm_lPFtha_L,vbm_lPFtha_R,Whole_fdg,Whole_amy,Whole_vbm,subGroup,Apoe
0,4589,130_S_4589,0,45.0000,AD,Negative,75.2000,26,2,1,...,0.2853,0.0935,0.3528,0.3983,0.2430,0.8197,1.2583,0.2402,AD_Neg,1
1,4732,116_S_4732,1,81.6000,AD,Negative,77.3000,21,1,1,...,0.5385,0.0828,0.3281,0.2940,0.2943,0.9133,1.1804,0.2340,AD_Neg,0
2,4827,011_S_4827,1,95.3000,AD,Negative,71.1000,26,1,1,...,0.4403,0.3413,0.3569,0.4441,0.4566,0.9241,1.3201,0.2320,AD_Neg,0
3,4845,011_S_4845,0,75.3000,AD,Negative,68.0000,22,2,1,...,0.3682,0.5485,0.2783,0.0906,0.2078,0.7852,1.2001,0.2099,AD_Neg,0
4,4859,068_S_4859,1,85.3000,AD,Negative,72.3000,24,1,1,...,0.4434,0.5965,0.4533,0.1352,0.2769,0.8460,1.5813,0.2216,AD_Neg,


In [3]:
regions_F_dat

,Networks,Regions,Modality,p_Group,p_SDB,p_inter,F_Group,F_SDB,F_inter,F_total
0,VISUAL,LH_VisCent_ExStr_1,av,0.0000,0.0200,0.8050,11.0150,5.5150,0.2180,19.5140
1,VISUAL,LH_VisCent_Striate_1,av,0.0000,0.0150,0.8100,16.2800,6.0020,0.2110,28.8570
2,VISUAL,LH_VisPeri_ExStrInf_1,av,0.0002,0.0180,0.6220,8.8170,5.7420,0.4760,21.6460
3,VISUAL,LH_VisPeri_ExStrSup_1,av,0.0005,0.0220,0.8490,8.0940,5.3770,0.1640,14.3800
4,VISUAL,LH_VisPeri_StriCal_1,av,0.0000,0.0130,0.8290,13.0500,6.2700,0.1880,25.5060
...,...,...,...,...,...,...,...,...,...,...
102,DAN,LH_DorsAttnA_SPL_1,vbm,0.0003,0.0440,0.9660,8.7050,4.1110,0.0350,22.3450
103,DAN,LH_DorsAttnA_TempOcc_1,vbm,0.0020,0.0400,0.7610,6.7260,4.2880,0.2730,14.9780
104,DAN,RH_DorsAttnA_ParOcc_1,vbm,0.0030,0.0350,0.1230,6.1750,4.5300,2.1230,16.6480
105,DAN,RH_DorsAttnA_TempOcc_1,vbm,0.0010,0.0340,0.7570,7.1370,4.5710,0.2790,15.5780


In [4]:
data2 = data.copy()
if 'Group*SDB' not in data2:
    data2.insert(5, 'Group*SDB', data2['Group']+'_'+data2['SDB'])

In [5]:
regions_F_dat

,Networks,Regions,Modality,p_Group,p_SDB,p_inter,F_Group,F_SDB,F_inter,F_total
0,VISUAL,LH_VisCent_ExStr_1,av,0.0000,0.0200,0.8050,11.0150,5.5150,0.2180,19.5140
1,VISUAL,LH_VisCent_Striate_1,av,0.0000,0.0150,0.8100,16.2800,6.0020,0.2110,28.8570
2,VISUAL,LH_VisPeri_ExStrInf_1,av,0.0002,0.0180,0.6220,8.8170,5.7420,0.4760,21.6460
3,VISUAL,LH_VisPeri_ExStrSup_1,av,0.0005,0.0220,0.8490,8.0940,5.3770,0.1640,14.3800
4,VISUAL,LH_VisPeri_StriCal_1,av,0.0000,0.0130,0.8290,13.0500,6.2700,0.1880,25.5060
...,...,...,...,...,...,...,...,...,...,...
102,DAN,LH_DorsAttnA_SPL_1,vbm,0.0003,0.0440,0.9660,8.7050,4.1110,0.0350,22.3450
103,DAN,LH_DorsAttnA_TempOcc_1,vbm,0.0020,0.0400,0.7610,6.7260,4.2880,0.2730,14.9780
104,DAN,RH_DorsAttnA_ParOcc_1,vbm,0.0030,0.0350,0.1230,6.1750,4.5300,2.1230,16.6480
105,DAN,RH_DorsAttnA_TempOcc_1,vbm,0.0010,0.0340,0.7570,7.1370,4.5710,0.2790,15.5780


In [6]:
regions = pd.read_csv('/home/mohammad/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_17Networks_order.csv')
regions.head()

,Value,ROI_Name
0,1,17Networks_LH_VisCent_ExStr_1
1,2,17Networks_LH_VisCent_ExStr_2
2,3,17Networks_LH_VisCent_Striate_1
3,4,17Networks_LH_VisCent_ExStr_3
4,5,17Networks_LH_VisPeri_ExStrInf_1


In [11]:
regions_F

,Regions
0,LH_VisCent_ExStr_1
1,LH_VisCent_Striate_1
2,LH_VisPeri_ExStrInf_1
3,LH_VisPeri_ExStrSup_1
4,LH_VisPeri_StriCal_1
...,...
102,LH_DorsAttnA_SPL_1
103,LH_DorsAttnA_TempOcc_1
104,RH_DorsAttnA_ParOcc_1
105,RH_DorsAttnA_TempOcc_1


In [20]:
## VBM
regions_F = regions_F_dat[regions_F_dat['Modality'] == 'vbm']
regions_F = regions_F[['Regions', 'F_total']]
regions_F = regions_F.dropna()
c = 0
for region in regions['ROI_Name']:
    #if region.find('Default') != -1:
    parcel = regions[regions['ROI_Name']==region]
    for region_F in regions_F["Regions"]:
        if region.find(region_F)>0:
            F_val = regions_F[regions_F['Regions']==region_F]
            F_val = float(F_val['F_total'])
            print(region, F_val)
    #print(int(parcel['Value']))
            name = 'vbm_map/'+region+'.nii.gz'
            parcel = int(parcel['Value'])
            fsl.fslmaths('/home/mohammad/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_17Networks_order_FSLMNI152_2mm.nii.gz').thr(parcel).uthr(parcel+0.5).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            if c == 0:
                fsl.fslmaths(reg_img).bin().mul(F_val).run('src.nii.gz')
                c += 1
                break
            source = image.load_img('src.nii.gz')
            fsl.fslmaths(reg_img).bin().mul(F_val).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            fsl.fslmaths(source).add(reg_img).run('src.nii.gz')
                
            
            
            #!fslmaths reg.nii.gz -mul $F_val $name
            break

17Networks_LH_DorsAttnA_TempOcc_1 14.978
17Networks_LH_DorsAttnA_SPL_1 22.345
17Networks_LH_SalVentAttnA_ParOper_1 28.627
17Networks_LH_SalVentAttnA_ParMed_1 23.67
17Networks_LH_SalVentAttnA_FrMed_1 17.13
17Networks_LH_ContA_IPS_1 42.455
17Networks_LH_ContB_PFClv_1 23.839
17Networks_LH_ContC_Cingp_1 20.553
17Networks_LH_DefaultB_IPL_1 28.719
17Networks_RH_SomMotB_Aud_1 37.344
17Networks_RH_DorsAttnA_TempOcc_1 15.578
17Networks_RH_DorsAttnA_ParOcc_1 16.648
17Networks_RH_DorsAttnB_PostC_1 32.427
17Networks_RH_SalVentAttnA_ParOper_1 18.869
17Networks_RH_SalVentAttnB_PFCl_1 13.514
17Networks_RH_LimbicB_OFC_1 35.033
17Networks_RH_LimbicA_TempPole_1 30.645
17Networks_RH_ContA_IPS_1 43.466
17Networks_RH_ContB_IPL_1 23.573
17Networks_RH_ContC_Cingp_1 14.109
17Networks_RH_DefaultA_IPL_1 24.975
17Networks_RH_DefaultC_PHC_1 19.537
17Networks_RH_TempPar_1 36.845
17Networks_RH_TempPar_2 38.501


In [21]:
## FDG
regions_F = regions_F_dat[regions_F_dat['Modality'] == 'fdg']
regions_F = regions_F[['Regions', 'F_total']]
regions_F = regions_F.dropna()
c = 0
for region in regions['ROI_Name']:
    #if region.find('Default') != -1:
    parcel = regions[regions['ROI_Name'] == region]
    for region_F in regions_F["Regions"]:
        if region.find(region_F)>0:
            F_val = regions_F[regions_F['Regions'] == region_F]
            F_val = float(F_val['F_total'])
            print(region, F_val)
    #print(int(parcel['Value']))
            #name = 'vbm_map/'+region+'.nii.gz'
            parcel = int(parcel['Value'])
            fsl.fslmaths('/home/mohammad/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_17Networks_order_FSLMNI152_2mm.nii.gz').thr(parcel).uthr(parcel+0.5).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            if c == 0:
                fsl.fslmaths(reg_img).bin().mul(F_val).run('src.nii.gz')
                c += 1
                break
            source = image.load_img('src.nii.gz')
            fsl.fslmaths(reg_img).bin().mul(F_val).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            fsl.fslmaths(source).add(reg_img).run('src.nii.gz')
                
            
            
            #!fslmaths reg.nii.gz -mul $F_val $name
            break

17Networks_LH_DorsAttnA_TempOcc_1 21.497
17Networks_LH_DorsAttnA_ParOcc_1 18.078
17Networks_LH_LimbicA_TempPole_2 34.575
17Networks_LH_DefaultB_Temp_1 43.594
17Networks_LH_DefaultB_Temp_2 18.195
17Networks_LH_DefaultB_IPL_1 25.951
17Networks_RH_DorsAttnA_TempOcc_1 24.372
17Networks_RH_DorsAttnA_ParOcc_1 25.05
17Networks_RH_DorsAttnA_SPL_1 16.17
17Networks_RH_DorsAttnB_PostC_1 19.03
17Networks_RH_SalVentAttnA_ParOper_1 28.612
17Networks_RH_SalVentAttnB_IPL_1 36.129
17Networks_RH_ContA_IPS_1 21.219
17Networks_RH_ContB_Temp_1 28.597
17Networks_RH_DefaultA_IPL_1 29.556
17Networks_RH_TempPar_1 36.968
17Networks_RH_TempPar_2 34.444


In [22]:
## AV
regions_F = regions_F_dat[regions_F_dat['Modality'] == 'av']
regions_F = regions_F[['Regions', 'F_total']]
regions_F = regions_F.dropna()
c = 0
for region in regions['ROI_Name']:
    #if region.find('Default') != -1:
    parcel = regions[regions['ROI_Name'] == region]
    for region_F in regions_F["Regions"]:
        if region.find(region_F)>0:
            F_val = regions_F[regions_F['Regions'] == region_F]
            F_val = float(F_val['F_total'])
            print(region, F_val)
    #print(int(parcel['Value']))
            name = 'av_map/'+region+'.nii.gz'
            parcel = int(parcel['Value'])
            fsl.fslmaths('/home/mohammad/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_17Networks_order_FSLMNI152_2mm.nii.gz').thr(parcel).uthr(parcel+0.5).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            if c == 0:
                fsl.fslmaths(reg_img).bin().mul(F_val).run('src.nii.gz')
                c += 1
                break
            source = image.load_img('src.nii.gz')
            fsl.fslmaths(reg_img).bin().mul(F_val).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            fsl.fslmaths(source).add(reg_img).run('src.nii.gz')
                
            
            
            #!fslmaths reg.nii.gz -mul $F_val $name
            break

17Networks_LH_VisCent_ExStr_1 19.514
17Networks_LH_VisCent_Striate_1 28.8570439
17Networks_LH_VisPeri_ExStrInf_1 21.646
17Networks_LH_VisPeri_StriCal_1 25.506
17Networks_LH_VisPeri_ExStrSup_1 14.38
17Networks_LH_SomMotA_2 12.138
17Networks_LH_SomMotB_Aud_1 25.856
17Networks_LH_SomMotB_S2_2 24.212
17Networks_LH_DorsAttnA_TempOcc_1 25.861
17Networks_LH_DorsAttnA_ParOcc_1 16.306
17Networks_LH_DorsAttnA_SPL_1 12.921
17Networks_LH_DorsAttnB_PostC_2 12.000234
17Networks_LH_DorsAttnB_FEF_1 13.2
17Networks_LH_SalVentAttnA_ParOper_1 15.771
17Networks_LH_SalVentAttnA_Ins_1 30.033
17Networks_LH_SalVentAttnA_Ins_2 17.3100311
17Networks_LH_SalVentAttnA_FrMed_1 15.451
17Networks_LH_LimbicB_OFC_1 28.323
17Networks_LH_LimbicA_TempPole_1 22.92
17Networks_LH_ContA_IPS_1 12.085
17Networks_LH_ContA_PFCl_2 12.928
17Networks_LH_ContC_pCun_1 24.497
17Networks_LH_ContC_pCun_2 22.623
17Networks_LH_DefaultA_pCunPCC_1 28.77929
17Networks_LH_DefaultB_Temp_1 25.532
17Networks_LH_DefaultB_Temp_2 26.781966
17Network

In [23]:
## Overlap
regions_F = regions_F_dat[regions_F_dat['Modality'] == 'Overlap']
regions_F = regions_F[['Regions', 'F_total']]
regions_F = regions_F.dropna()
c = 0
for region in regions['ROI_Name']:
    #if region.find('Default') != -1:
    parcel = regions[regions['ROI_Name'] == region]
    for region_F in regions_F["Regions"]:
        if region.find(region_F)>0:
            F_val = regions_F[regions_F['Regions'] == region_F]
            F_val = float(F_val['F_total'])
            print(region, F_val)
    #print(int(parcel['Value']))
            name = 'av_map/'+region+'.nii.gz'
            parcel = int(parcel['Value'])
            fsl.fslmaths('/home/mohammad/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_17Networks_order_FSLMNI152_2mm.nii.gz').thr(parcel).uthr(parcel+0.5).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            if c == 0:
                fsl.fslmaths(reg_img).bin().mul(F_val).run('src.nii.gz')
                c += 1
                break
            source = image.load_img('src.nii.gz')
            fsl.fslmaths(reg_img).bin().mul(F_val).run('reg.nii.gz')
            reg_img = image.load_img('reg.nii.gz')
            fsl.fslmaths(source).add(reg_img).run('src.nii.gz')
                
            
            
            #!fslmaths reg.nii.gz -mul $F_val $name
            break

17Networks_LH_DorsAttnA_TempOcc_1 30.0
17Networks_LH_DefaultB_IPL_1 30.0
17Networks_RH_DorsAttnA_TempOcc_1 30.0
17Networks_RH_DorsAttnA_ParOcc_1 30.0
17Networks_RH_DorsAttnB_PostC_1 30.0
17Networks_RH_ContA_IPS_1 30.0
17Networks_RH_TempPar_1 30.0
17Networks_RH_TempPar_2 30.0
